In [1]:
import sys
import random
import copy
import operator
from itertools import chain
import pandas as pd
import numpy as np
from typing_extensions import final
from pysat.formula import CNF
import os
import math
from itertools import chain
import random
import time

# Heuristics

In [4]:
def JW(sigma):
    jw_counts = {}
    for clause in sigma:
        for literal in clause:
            if literal in jw_counts:
                jw_counts[literal] += 2 ** - len(clause)
            else:
                jw_counts[literal] = 2 ** - len(clause)

    max_lit = max(jw_counts, key=jw_counts.get)

    return max_lit

def DLCS(sigma):                               # Dynamic Largest Combined Sum
    flat_cnf = [abs(p) for c in sigma for p in c]         # Flatten the cnf
    return max(flat_cnf, key=flat_cnf.count)            # Return the most frequent predicate

def DLCS_2(cnf):
    weights = {}
    for clause in cnf:
        for literal in clause:
            if literal in weights:
                weights[literal] += 1
            else:
                weights[literal] = 1
    return max(weights, key=weights.get)

def mrv(removed_literals, unique_variables):
    """
    Returns the literal with the highest score based upon Minimum Remaining
    Values (MRV), on which the dpll2 should branch on next
    """
    to_choose_dict_2 ={}
    row_dict = {i:0 for i in range(1,10)}
    column_dict = {i:0 for i in range(1,10)}

    for removed_literal in removed_literals:
        row = abs(removed_literal) // 10**2 %10
        column = abs(removed_literal) // 10 % 10
        #print(f"Row {row}, and column {column}")
        row_dict[row] = row_dict[row] +1
        column_dict[column] = column_dict[column] +1

    for unique_variable in unique_variables:
        var_row = abs(unique_variable) // 10**2 %10
        var_column = abs(unique_variable) // 10 % 10
        to_choose_dict_2[unique_variable] = row_dict[var_row] + column_dict[var_column]
    max(to_choose_dict_2, key=to_choose_dict_2.get)

    return max(to_choose_dict_2, key=to_choose_dict_2.get)


# Initializing sudoku 

In [6]:

def read_DIMACS(path):
    """
    Takes a DIMACS input file and returns a list of clauses
    """
    return CNF(from_file=path).clauses

def line2DIMACS(sudoku):
    """
    Takes one line of a sudoku file and converts it to a DIMACS line
    for variables that are true for this sudoku
    """
    input_mapping = {'0':0,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,
    'A': 10, 'B': 11, "C": 12, 'D': 13, 'E': 14, 'F': 15, 'G': 16}

    inverse_input_mapping = {v:k for k, v in input_mapping.items()}

    input_len = len(sudoku)
    sudoku_size = int(math.pow(input_len, 0.5))
    clause_list = []
    for row in range(sudoku_size):
        for column in range(sudoku_size):
            current_line_index = column + sudoku_size * row
            cell_value = sudoku[current_line_index]
            if cell_value != '.':
                clause_list.append(f"{inverse_input_mapping[row+1]}{inverse_input_mapping[column+1]}{cell_value} 0")
    return clause_list, sudoku_size

def line2CNF(line):
    """
    Converts the line out of the line2DIMACS function and returns it as a list
    of clauses of which variables are true in this sudoku
    """
    cnf_list, sudoku_size = line2DIMACS(line)
    return CNF(from_string='\n'.join(cnf_list)).clauses, cnf_list, sudoku_size


# Accesory functions

In [7]:

def simplify(rules, literal):
    """
    Simplifies the rules list of clauses by removing the true clauses and
    remove negations of true values out of a clause of a given value
    """
    for rule_clause in list(rules):
        # remove clause when true variable is in that clause
        if literal in rule_clause:
            rules.remove(rule_clause)

        # remove negative value from the clause when present
        neg_literal = 0 - literal
        if neg_literal in rule_clause:
            rule_clause.remove(neg_literal)

    return rules

def find_pure_literals(sigma):
    pure_lits = []
    merged = set(chain.from_iterable(sigma))

    for lit in merged:
        if -lit not in merged:
            pure_lits.append(lit)

    return pure_lits


def is_tautology(clause):
    neg_clause = {literal * -1 for literal in clause}
    diff = clause - neg_clause
    tautology = clause - diff

    if tautology:
        return tautology
    else:
        return None

def remove_negative(rules):
    no_neg = []
    for lit in rules[1]:
        if lit > 0:
            no_neg.append(lit)
    return no_neg



def remove_unit_clauses(rules):
    """
    Removes unit clauses from given list of rules
    """
    # get all unit clauses
    unit_clauses = []
    for clause in rules:
        if len(clause) == 1:
            literal = clause[0]
            unit_clauses.append(literal)

    # make sure unit literals occur only once
    unit_clauses = list(set(unit_clauses))

    # simplify rules based on all unit clauses
    if len(unit_clauses) != 0:
        for literal in unit_clauses:
            rules = simplify(rules, literal)

    return rules, unit_clauses


def pretty_print_sudoku(solution, size):

    # Init board
    board = [['.' for x in range(size)] for y in range(size)]

    for literal in solution:
        # We need a true literal as well as one that is positive
            number = str(literal - 110)

            # Prepend a zero so that we have 3 character string

            diff = 3 - len(number)
            number = "0" * diff + number
            row = int(number[0])
            col = int(number[1])
            val = int(number[2])

            board[row][col] = val

    builder = ""
    for row in board:
        builder += str(row) + "\n"

    print(builder)

# SAT with DPLL

In [8]:

def SAT(rules):
    """
    Returns true when rules is empty, which means that the problem has been
    solved
    """
    if len(rules) == 0:
        return True

    # check if clause is empty
    for clause in rules:
        if len(clause) == 0:
            return False

    return None


def solve(rules, solution, heuristic):
    """
    Solves the rest of the SAT problem until SAT returns True or False. Returning
    True means that there are no rules left to simplify and thus that the problem
    is satisfied. Returning false means that there is no solution to this problem
    """
    global stats
    # check if the problem is satisfied
    if SAT(rules) != None:
        return SAT(rules), solution

    # create a list of all literals that are left in the rules
    unique_literals = list(set(chain.from_iterable(rules)))
    copy_rules = copy.deepcopy(rules)

    # implement heuristics
    if heuristic == '1':
        picked_literal = random.choice(unique_literals)
    if heuristic == '2':
        picked_literal = mrv(solution, unique_literals)
    if heuristic == '3':
        picked_literal = JW(rules)
    if heuristic == '4': 
        picked_literal = DLCS(rules)
    if heuristic == '5':
        picked_literal = DLCS_2(rules)
    
    new_rules = simplify(copy_rules, picked_literal)
    new_solution = solution.copy()
    new_solution.append(picked_literal)

    # filter out unit clauses again
    new_rules, removed_literals = remove_unit_clauses(new_rules)
    new_solution.extend(removed_literals)

    # check if it worked. If so, continue on this branch
    result = solve(new_rules, new_solution, heuristic)
    if result[0]:
        return result

    #print("backbranch")
    stats['backs'] =stats['backs'] + 1
    picked_literal = -picked_literal
    new_rules = simplify(rules, picked_literal)
    new_solution = solution.copy()
    new_solution.append(picked_literal)

    # filter out unit clauses again
    new_rules, removed_literals = remove_unit_clauses(new_rules)
    new_solution.extend(removed_literals)

    # add picked literal to the solution
    return solve(new_rules, new_solution, heuristic)

def dpll2(rules, heuristic):
    """
    Remove unit clauses and start solving the problem. Returns True when problem
    is satisfied and its solution. Returns false and a counter example when problem
    can not be satisfied
    """

    # check if the problem is satisfied
    if SAT(rules) != None:
        return SAT(rules)

    # remove unit clauses
    rules, removed_clauses = remove_unit_clauses(rules)

    return solve(rules, removed_clauses, heuristic)

# Experiment function

In [16]:
def run_sat(heuristic,sudoku_nr,file='damnhard.sdk.txt' ):
    """
    Reads input arguments and sets up the right rules for the SAT problem
    """
    # stores each sudoku example in a list
    with open(file) as f:
        sudoku_list = f.readlines()

    # get the CNF of the current sudoku example (sudoku_setup_CNF)
    sudoku_setup_CNF, sudoku_setup_DIMACS, sudoku_size = line2CNF(sudoku_list[sudoku_nr])

    # create rules and add setup rules as unit clauses
    rules = read_DIMACS(os.path.join('sudokus/sudoku-rules-9x9.txt'))
    rules.extend(sudoku_setup_CNF)

    # start the dpll2 algorithm
    final_results = dpll2(rules, heuristic)

## Solving hard sudokus

In [22]:
test_mrv = []
heuristics = ['2']
for heuristic in heuristics:
    for sudoku_nr in range(35):
        stats = {"backs":0, "time":0}
        print(f'Initiating sudoku {sudoku_nr}')
        start_time = time.time()
        run_sat(heuristic,sudoku_nr,'sudokus/damnhard.sdk.txt')
        stop_time = time.time()
        stats['time'] = stop_time - start_time
        test_mrv.append([sudoku_nr,'MRV',stats['time'],stats['backs']])
        print(f'Sudoku {sudoku_nr}, stats: {stats}')



Initiating sudoku 0
Sudoku 0, stats: {'backs': 162, 'time': 4.187521696090698}
Initiating sudoku 1
Sudoku 1, stats: {'backs': 234, 'time': 6.517086505889893}
Initiating sudoku 2
Sudoku 2, stats: {'backs': 31, 'time': 3.9658634662628174}
Initiating sudoku 3
Sudoku 3, stats: {'backs': 46, 'time': 3.6910605430603027}
Initiating sudoku 4
Sudoku 4, stats: {'backs': 77, 'time': 4.748439073562622}
Initiating sudoku 5
Sudoku 5, stats: {'backs': 179, 'time': 4.976777076721191}
Initiating sudoku 6
Sudoku 6, stats: {'backs': 245, 'time': 5.895761728286743}
Initiating sudoku 7
Sudoku 7, stats: {'backs': 27, 'time': 3.688591718673706}
Initiating sudoku 8
Sudoku 8, stats: {'backs': 466, 'time': 8.753648281097412}
Initiating sudoku 9
Sudoku 9, stats: {'backs': 294, 'time': 5.9879090785980225}
Initiating sudoku 10
Sudoku 10, stats: {'backs': 611, 'time': 9.89199686050415}
Initiating sudoku 11
Sudoku 11, stats: {'backs': 82, 'time': 4.077722072601318}
Initiating sudoku 12
Sudoku 12, stats: {'backs': 42

In [23]:
test_random = []
heuristics = ['1']
for heuristic in heuristics:
    for sudoku_nr in range(35):
        stats = {"backs":0, "time":0}
        print(f'Initiating sudoku {sudoku_nr}')
        start_time = time.time()
        run_sat(heuristic,sudoku_nr,'sudokus/damnhard.sdk.txt')
        stop_time = time.time()
        stats['time'] = stop_time - start_time
        test_random.append([sudoku_nr,'Random',stats['time'],stats['backs']])
        print(f'Sudoku {sudoku_nr}, stats: {stats}')



Initiating sudoku 0
Sudoku 0, stats: {'backs': 648, 'time': 6.186001539230347}
Initiating sudoku 1
Sudoku 1, stats: {'backs': 1388, 'time': 9.390000343322754}
Initiating sudoku 2
Sudoku 2, stats: {'backs': 2621, 'time': 20.762996435165405}
Initiating sudoku 3
Sudoku 3, stats: {'backs': 1737, 'time': 16.707002639770508}
Initiating sudoku 4
Sudoku 4, stats: {'backs': 746, 'time': 6.663997650146484}
Initiating sudoku 5
Sudoku 5, stats: {'backs': 110, 'time': 2.216994285583496}
Initiating sudoku 6
Sudoku 6, stats: {'backs': 1979, 'time': 19.389999866485596}
Initiating sudoku 7
Sudoku 7, stats: {'backs': 1584, 'time': 15.712000370025635}
Initiating sudoku 8
Sudoku 8, stats: {'backs': 1422, 'time': 13.96899962425232}
Initiating sudoku 9
Sudoku 9, stats: {'backs': 31, 'time': 2.5590004920959473}
Initiating sudoku 10
Sudoku 10, stats: {'backs': 2738, 'time': 18.98800015449524}
Initiating sudoku 11
Sudoku 11, stats: {'backs': 927, 'time': 8.175866603851318}
Initiating sudoku 12
Sudoku 12, stats

In [24]:
test_jw = []
heuristics = ['3']
for heuristic in heuristics:
    for sudoku_nr in range(35):
        stats = {"backs":0, "time":0}
        print(f'Initiating sudoku {sudoku_nr}')
        start_time = time.time()
        run_sat(heuristic,sudoku_nr,'sudokus/damnhard.sdk.txt')
        stop_time = time.time()
        stats['time'] = stop_time - start_time
        test_jw.append([sudoku_nr,'JW',stats['time'],stats['backs']])
        print(f'Sudoku {sudoku_nr}, stats: {stats}')


Initiating sudoku 0
Sudoku 0, stats: {'backs': 61543, 'time': 225.31210231781006}
Initiating sudoku 1
Sudoku 1, stats: {'backs': 12624, 'time': 44.97100019454956}
Initiating sudoku 2
Sudoku 2, stats: {'backs': 60733, 'time': 240.26217460632324}
Initiating sudoku 3
Sudoku 3, stats: {'backs': 242326, 'time': 969.9982295036316}
Initiating sudoku 4
Sudoku 4, stats: {'backs': 79910, 'time': 352.40011382102966}
Initiating sudoku 5
Sudoku 5, stats: {'backs': 141053, 'time': 575.7523510456085}
Initiating sudoku 6
Sudoku 6, stats: {'backs': 53517, 'time': 233.60310077667236}
Initiating sudoku 7
Sudoku 7, stats: {'backs': 47408, 'time': 212.52291107177734}
Initiating sudoku 8
Sudoku 8, stats: {'backs': 136113, 'time': 575.9251983165741}
Initiating sudoku 9
Sudoku 9, stats: {'backs': 9014, 'time': 36.60704493522644}
Initiating sudoku 10
Sudoku 10, stats: {'backs': 58936, 'time': 244.4899971485138}
Initiating sudoku 11
Sudoku 11, stats: {'backs': 69733, 'time': 561.7353847026825}
Initiating sudoku

In [25]:
test_dcls = []
heuristics = ['4']
for heuristic in heuristics:
    for sudoku_nr in range(35):
        stats = {"backs":0, "time":0}
        print(f'Initiating sudoku {sudoku_nr}')
        start_time = time.time()
        run_sat(heuristic,sudoku_nr,'sudokus/damnhard.sdk.txt')
        stop_time = time.time()
        stats['time'] = stop_time - start_time
        test_dcls.append([sudoku_nr,'DCLS',stats['time'],stats['backs']])
        print(f'Sudoku {sudoku_nr}, stats: {stats}')


Initiating sudoku 0
Sudoku 0, stats: {'backs': 9476, 'time': 213.55999898910522}
Initiating sudoku 1
Sudoku 1, stats: {'backs': 2216, 'time': 59.022003412246704}
Initiating sudoku 2
Sudoku 2, stats: {'backs': 87376, 'time': 2011.6091351509094}
Initiating sudoku 3
Sudoku 3, stats: {'backs': 28333, 'time': 715.3300018310547}
Initiating sudoku 4
Sudoku 4, stats: {'backs': 56358, 'time': 1369.7862243652344}
Initiating sudoku 5
Sudoku 5, stats: {'backs': 57826, 'time': 1490.2557094097137}
Initiating sudoku 6
Sudoku 6, stats: {'backs': 21140, 'time': 568.5151264667511}
Initiating sudoku 7
Sudoku 7, stats: {'backs': 3647, 'time': 71.08008217811584}
Initiating sudoku 8
Sudoku 8, stats: {'backs': 28087, 'time': 773.3431413173676}
Initiating sudoku 9
Sudoku 9, stats: {'backs': 10898, 'time': 278.54100728034973}
Initiating sudoku 10
Sudoku 10, stats: {'backs': 52490, 'time': 1126.7973561286926}
Initiating sudoku 11
Sudoku 11, stats: {'backs': 4803, 'time': 132.9590003490448}
Initiating sudoku 12


In [26]:
test_dcls_2 = []
heuristics = ['5']
for heuristic in heuristics:
    for sudoku_nr in range(35):
        stats = {"backs":0, "time":0}
        print(f'Initiating sudoku {sudoku_nr}')
        start_time = time.time()
        run_sat(heuristic,sudoku_nr,'sudokus/damnhard.sdk.txt')
        stop_time = time.time()
        stats['time'] = stop_time - start_time
        test_dcls_2.append([sudoku_nr,'DCLS_2',stats['time'],stats['backs']])
        print(f'Sudoku {sudoku_nr}, stats: {stats}')


Initiating sudoku 0
Sudoku 0, stats: {'backs': 79409, 'time': 292.4050467014313}
Initiating sudoku 1
Sudoku 1, stats: {'backs': 13730, 'time': 47.875998973846436}
Initiating sudoku 2
Sudoku 2, stats: {'backs': 66377, 'time': 255.9670648574829}
Initiating sudoku 3
Sudoku 3, stats: {'backs': 258859, 'time': 1003.3402218818665}
Initiating sudoku 4
Sudoku 4, stats: {'backs': 89552, 'time': 379.8290011882782}
Initiating sudoku 5
Sudoku 5, stats: {'backs': 142066, 'time': 555.3160486221313}
Initiating sudoku 6
Sudoku 6, stats: {'backs': 57092, 'time': 233.39199948310852}
Initiating sudoku 7
Sudoku 7, stats: {'backs': 3913, 'time': 11.32300066947937}
Initiating sudoku 8
Sudoku 8, stats: {'backs': 146741, 'time': 611.3930020332336}
Initiating sudoku 9
Sudoku 9, stats: {'backs': 9692, 'time': 38.25200033187866}
Initiating sudoku 10
Sudoku 10, stats: {'backs': 6374, 'time': 24.293999671936035}
Initiating sudoku 11
Sudoku 11, stats: {'backs': 78184, 'time': 299.08208894729614}
Initiating sudoku 1

In [27]:
import pandas as pd

df_1 = pd.DataFrame(test_mrv)
df_1.to_csv('test_mrv.csv')
df_2 = pd.DataFrame(test_random)
df_2.to_csv('test_random.csv')
df_3 = pd.DataFrame(test_jw)
df_3.to_csv('test_jw.csv')
df_4 = pd.DataFrame(test_dcls)
df_4.to_csv('test_dcls.csv')
df_5 = pd.DataFrame(test_dcls_2)
df_5.to_csv('test_dcls_2.csv')

## Solving 1000 sudokus

In [28]:
test_mrv_1000 = []
heuristics = ['2']
for heuristic in heuristics:
    for sudoku_nr in range(999):
        stats = {"backs":0, "time":0}
        print(f'Initiating sudoku {sudoku_nr}')
        start_time = time.time()
        run_sat(heuristic,sudoku_nr,'sudokus/1000 sudokus.txt')
        stop_time = time.time()
        stats['time'] = stop_time - start_time
        test_mrv_1000.append([sudoku_nr,'MRV',stats['time'],stats['backs']])
        print(f'Sudoku {sudoku_nr}, stats: {stats}')


Initiating sudoku 0
Sudoku 0, stats: {'backs': 22, 'time': 2.1270010471343994}
Initiating sudoku 1
Sudoku 1, stats: {'backs': 19, 'time': 2.200000047683716}
Initiating sudoku 2
Sudoku 2, stats: {'backs': 22, 'time': 2.157999277114868}
Initiating sudoku 3
Sudoku 3, stats: {'backs': 32, 'time': 2.25}
Initiating sudoku 4
Sudoku 4, stats: {'backs': 25, 'time': 2.2799980640411377}
Initiating sudoku 5
Sudoku 5, stats: {'backs': 41, 'time': 2.2030022144317627}
Initiating sudoku 6
Sudoku 6, stats: {'backs': 17, 'time': 2.129000425338745}
Initiating sudoku 7
Sudoku 7, stats: {'backs': 35, 'time': 2.118000030517578}
Initiating sudoku 8
Sudoku 8, stats: {'backs': 5, 'time': 2.1969974040985107}
Initiating sudoku 9
Sudoku 9, stats: {'backs': 19, 'time': 2.0960025787353516}
Initiating sudoku 10
Sudoku 10, stats: {'backs': 130, 'time': 2.7419989109039307}
Initiating sudoku 11
Sudoku 11, stats: {'backs': 21, 'time': 2.1793673038482666}
Initiating sudoku 12
Sudoku 12, stats: {'backs': 32, 'time': 2.384

Sudoku 102, stats: {'backs': 15, 'time': 2.202000141143799}
Initiating sudoku 103
Sudoku 103, stats: {'backs': 8, 'time': 2.2849996089935303}
Initiating sudoku 104
Sudoku 104, stats: {'backs': 9, 'time': 2.1680002212524414}
Initiating sudoku 105
Sudoku 105, stats: {'backs': 18, 'time': 2.2029991149902344}
Initiating sudoku 106
Sudoku 106, stats: {'backs': 8, 'time': 2.1790003776550293}
Initiating sudoku 107
Sudoku 107, stats: {'backs': 6, 'time': 2.1600000858306885}
Initiating sudoku 108
Sudoku 108, stats: {'backs': 65, 'time': 2.6640002727508545}
Initiating sudoku 109
Sudoku 109, stats: {'backs': 32, 'time': 2.2950003147125244}
Initiating sudoku 110
Sudoku 110, stats: {'backs': 20, 'time': 2.182013750076294}
Initiating sudoku 111
Sudoku 111, stats: {'backs': 11, 'time': 2.183986186981201}
Initiating sudoku 112
Sudoku 112, stats: {'backs': 138, 'time': 2.740996837615967}
Initiating sudoku 113
Sudoku 113, stats: {'backs': 31, 'time': 2.336002826690674}
Initiating sudoku 114
Sudoku 114, 

Sudoku 202, stats: {'backs': 36, 'time': 1.19700026512146}
Initiating sudoku 203
Sudoku 203, stats: {'backs': 22, 'time': 2.3209996223449707}
Initiating sudoku 204
Sudoku 204, stats: {'backs': 28, 'time': 2.2979979515075684}
Initiating sudoku 205
Sudoku 205, stats: {'backs': 11, 'time': 2.2210023403167725}
Initiating sudoku 206
Sudoku 206, stats: {'backs': 40, 'time': 2.3919990062713623}
Initiating sudoku 207
Sudoku 207, stats: {'backs': 70, 'time': 2.6329989433288574}
Initiating sudoku 208
Sudoku 208, stats: {'backs': 40, 'time': 2.3450005054473877}
Initiating sudoku 209
Sudoku 209, stats: {'backs': 34, 'time': 2.30501127243042}
Initiating sudoku 210
Sudoku 210, stats: {'backs': 11, 'time': 2.2460038661956787}
Initiating sudoku 211
Sudoku 211, stats: {'backs': 20, 'time': 2.2430121898651123}
Initiating sudoku 212
Sudoku 212, stats: {'backs': 9, 'time': 2.288970947265625}
Initiating sudoku 213
Sudoku 213, stats: {'backs': 31, 'time': 2.336998462677002}
Initiating sudoku 214
Sudoku 214,

Sudoku 302, stats: {'backs': 19, 'time': 2.339024782180786}
Initiating sudoku 303
Sudoku 303, stats: {'backs': 7, 'time': 2.1989729404449463}
Initiating sudoku 304
Sudoku 304, stats: {'backs': 31, 'time': 2.374999523162842}
Initiating sudoku 305
Sudoku 305, stats: {'backs': 15, 'time': 1.198002576828003}
Initiating sudoku 306
Sudoku 306, stats: {'backs': 28, 'time': 2.265000581741333}
Initiating sudoku 307
Sudoku 307, stats: {'backs': 23, 'time': 2.3729982376098633}
Initiating sudoku 308
Sudoku 308, stats: {'backs': 28, 'time': 2.3399977684020996}
Initiating sudoku 309
Sudoku 309, stats: {'backs': 12, 'time': 2.231003522872925}
Initiating sudoku 310
Sudoku 310, stats: {'backs': 12, 'time': 2.225996971130371}
Initiating sudoku 311
Sudoku 311, stats: {'backs': 44, 'time': 2.323000907897949}
Initiating sudoku 312
Sudoku 312, stats: {'backs': 9, 'time': 2.3720011711120605}
Initiating sudoku 313
Sudoku 313, stats: {'backs': 18, 'time': 2.2990000247955322}
Initiating sudoku 314
Sudoku 314, s

Sudoku 402, stats: {'backs': 30, 'time': 2.2240028381347656}
Initiating sudoku 403
Sudoku 403, stats: {'backs': 41, 'time': 2.2719969749450684}
Initiating sudoku 404
Sudoku 404, stats: {'backs': 31, 'time': 2.332003593444824}
Initiating sudoku 405
Sudoku 405, stats: {'backs': 48, 'time': 2.3939976692199707}
Initiating sudoku 406
Sudoku 406, stats: {'backs': 8, 'time': 2.346001148223877}
Initiating sudoku 407
Sudoku 407, stats: {'backs': 21, 'time': 2.240018129348755}
Initiating sudoku 408
Sudoku 408, stats: {'backs': 14, 'time': 2.259981870651245}
Initiating sudoku 409
Sudoku 409, stats: {'backs': 21, 'time': 2.2860004901885986}
Initiating sudoku 410
Sudoku 410, stats: {'backs': 4, 'time': 2.189000129699707}
Initiating sudoku 411
Sudoku 411, stats: {'backs': 34, 'time': 2.4649996757507324}
Initiating sudoku 412
Sudoku 412, stats: {'backs': 20, 'time': 1.1730003356933594}
Initiating sudoku 413
Sudoku 413, stats: {'backs': 39, 'time': 2.273998975753784}
Initiating sudoku 414
Sudoku 414, 

Sudoku 502, stats: {'backs': 16, 'time': 2.260998249053955}
Initiating sudoku 503
Sudoku 503, stats: {'backs': 58, 'time': 2.590999126434326}
Initiating sudoku 504
Sudoku 504, stats: {'backs': 70, 'time': 2.621002674102783}
Initiating sudoku 505
Sudoku 505, stats: {'backs': 15, 'time': 2.254000663757324}
Initiating sudoku 506
Sudoku 506, stats: {'backs': 20, 'time': 2.222999095916748}
Initiating sudoku 507
Sudoku 507, stats: {'backs': 36, 'time': 2.3310022354125977}
Initiating sudoku 508
Sudoku 508, stats: {'backs': 13, 'time': 2.19899845123291}
Initiating sudoku 509
Sudoku 509, stats: {'backs': 34, 'time': 2.4039974212646484}
Initiating sudoku 510
Sudoku 510, stats: {'backs': 19, 'time': 2.2970001697540283}
Initiating sudoku 511
Sudoku 511, stats: {'backs': 24, 'time': 2.3329997062683105}
Initiating sudoku 512
Sudoku 512, stats: {'backs': 10, 'time': 2.281002998352051}
Initiating sudoku 513
Sudoku 513, stats: {'backs': 120, 'time': 2.7079970836639404}
Initiating sudoku 514
Sudoku 514,

Sudoku 602, stats: {'backs': 46, 'time': 2.407996892929077}
Initiating sudoku 603
Sudoku 603, stats: {'backs': 16, 'time': 2.358999729156494}
Initiating sudoku 604
Sudoku 604, stats: {'backs': 23, 'time': 2.3880035877227783}
Initiating sudoku 605
Sudoku 605, stats: {'backs': 32, 'time': 2.322998523712158}
Initiating sudoku 606
Sudoku 606, stats: {'backs': 49, 'time': 2.388002395629883}
Initiating sudoku 607
Sudoku 607, stats: {'backs': 41, 'time': 2.3659985065460205}
Initiating sudoku 608
Sudoku 608, stats: {'backs': 44, 'time': 2.3559978008270264}
Initiating sudoku 609
Sudoku 609, stats: {'backs': 21, 'time': 2.355001211166382}
Initiating sudoku 610
Sudoku 610, stats: {'backs': 21, 'time': 2.3519983291625977}
Initiating sudoku 611
Sudoku 611, stats: {'backs': 57, 'time': 2.4029998779296875}
Initiating sudoku 612
Sudoku 612, stats: {'backs': 55, 'time': 2.522000789642334}
Initiating sudoku 613
Sudoku 613, stats: {'backs': 10, 'time': 2.1880154609680176}
Initiating sudoku 614
Sudoku 614

Sudoku 702, stats: {'backs': 46, 'time': 2.4730002880096436}
Initiating sudoku 703
Sudoku 703, stats: {'backs': 35, 'time': 2.344003200531006}
Initiating sudoku 704
Sudoku 704, stats: {'backs': 11, 'time': 2.4169998168945312}
Initiating sudoku 705
Sudoku 705, stats: {'backs': 10, 'time': 2.2849998474121094}
Initiating sudoku 706
Sudoku 706, stats: {'backs': 30, 'time': 2.2970006465911865}
Initiating sudoku 707
Sudoku 707, stats: {'backs': 11, 'time': 2.30299711227417}
Initiating sudoku 708
Sudoku 708, stats: {'backs': 35, 'time': 2.438002109527588}
Initiating sudoku 709
Sudoku 709, stats: {'backs': 42, 'time': 2.492274522781372}
Initiating sudoku 710
Sudoku 710, stats: {'backs': 27, 'time': 2.293003559112549}
Initiating sudoku 711
Sudoku 711, stats: {'backs': 20, 'time': 2.306999444961548}
Initiating sudoku 712
Sudoku 712, stats: {'backs': 11, 'time': 2.2689969539642334}
Initiating sudoku 713
Sudoku 713, stats: {'backs': 9, 'time': 2.3110032081604004}
Initiating sudoku 714
Sudoku 714, 

Sudoku 802, stats: {'backs': 13, 'time': 2.3050284385681152}
Initiating sudoku 803
Sudoku 803, stats: {'backs': 31, 'time': 2.4739720821380615}
Initiating sudoku 804
Sudoku 804, stats: {'backs': 59, 'time': 2.4489998817443848}
Initiating sudoku 805
Sudoku 805, stats: {'backs': 24, 'time': 2.3489978313446045}
Initiating sudoku 806
Sudoku 806, stats: {'backs': 48, 'time': 2.41499924659729}
Initiating sudoku 807
Sudoku 807, stats: {'backs': 11, 'time': 2.2380006313323975}
Initiating sudoku 808
Sudoku 808, stats: {'backs': 70, 'time': 2.60400128364563}
Initiating sudoku 809
Sudoku 809, stats: {'backs': 40, 'time': 2.4239985942840576}
Initiating sudoku 810
Sudoku 810, stats: {'backs': 34, 'time': 2.3680012226104736}
Initiating sudoku 811
Sudoku 811, stats: {'backs': 84, 'time': 2.5829997062683105}
Initiating sudoku 812
Sudoku 812, stats: {'backs': 90, 'time': 2.710001230239868}
Initiating sudoku 813
Sudoku 813, stats: {'backs': 12, 'time': 2.309007406234741}
Initiating sudoku 814
Sudoku 814

Sudoku 902, stats: {'backs': 98, 'time': 2.764998197555542}
Initiating sudoku 903
Sudoku 903, stats: {'backs': 58, 'time': 2.494001865386963}
Initiating sudoku 904
Sudoku 904, stats: {'backs': 25, 'time': 2.5629982948303223}
Initiating sudoku 905
Sudoku 905, stats: {'backs': 50, 'time': 2.5710031986236572}
Initiating sudoku 906
Sudoku 906, stats: {'backs': 35, 'time': 2.4599974155426025}
Initiating sudoku 907
Sudoku 907, stats: {'backs': 71, 'time': 2.5640006065368652}
Initiating sudoku 908
Sudoku 908, stats: {'backs': 42, 'time': 2.5040013790130615}
Initiating sudoku 909
Sudoku 909, stats: {'backs': 27, 'time': 2.444999933242798}
Initiating sudoku 910
Sudoku 910, stats: {'backs': 85, 'time': 2.6779987812042236}
Initiating sudoku 911
Sudoku 911, stats: {'backs': 9, 'time': 2.2229998111724854}
Initiating sudoku 912
Sudoku 912, stats: {'backs': 6, 'time': 2.2499985694885254}
Initiating sudoku 913
Sudoku 913, stats: {'backs': 42, 'time': 2.5180256366729736}
Initiating sudoku 914
Sudoku 91

In [29]:
test_jw_1000 = []
heuristics = ['3']
for heuristic in heuristics:
    for sudoku_nr in range(999):
        stats = {"backs":0, "time":0}
        print(f'Initiating sudoku {sudoku_nr}')
        start_time = time.time()
        run_sat(heuristic,sudoku_nr,'sudokus/1000 sudokus.txt')
        stop_time = time.time()
        stats['time'] = stop_time - start_time
        test_jw_1000.append([sudoku_nr,'MRV',stats['time'],stats['backs']])
        print(f'Sudoku {sudoku_nr}, stats: {stats}')

Initiating sudoku 0
Sudoku 0, stats: {'backs': 38, 'time': 1.3370006084442139}
Initiating sudoku 1
Sudoku 1, stats: {'backs': 0, 'time': 1.1599962711334229}
Initiating sudoku 2
Sudoku 2, stats: {'backs': 431, 'time': 2.308000087738037}
Initiating sudoku 3
Sudoku 3, stats: {'backs': 43, 'time': 1.26900315284729}
Initiating sudoku 4
Sudoku 4, stats: {'backs': 0, 'time': 1.181018590927124}
Initiating sudoku 5
Sudoku 5, stats: {'backs': 532, 'time': 2.593979835510254}
Initiating sudoku 6
Sudoku 6, stats: {'backs': 39, 'time': 1.3079991340637207}
Initiating sudoku 7
Sudoku 7, stats: {'backs': 8, 'time': 1.2800025939941406}
Initiating sudoku 8
Sudoku 8, stats: {'backs': 0, 'time': 1.2190003395080566}
Initiating sudoku 9
Sudoku 9, stats: {'backs': 641, 'time': 3.177999496459961}
Initiating sudoku 10
Sudoku 10, stats: {'backs': 5481, 'time': 20.623000144958496}
Initiating sudoku 11
Sudoku 11, stats: {'backs': 7, 'time': 1.312000036239624}
Initiating sudoku 12
Sudoku 12, stats: {'backs': 61, 't

Sudoku 102, stats: {'backs': 203, 'time': 1.9093472957611084}
Initiating sudoku 103
Sudoku 103, stats: {'backs': 136, 'time': 1.5430006980895996}
Initiating sudoku 104
Sudoku 104, stats: {'backs': 0, 'time': 1.231001853942871}
Initiating sudoku 105
Sudoku 105, stats: {'backs': 5, 'time': 1.3909974098205566}
Initiating sudoku 106
Sudoku 106, stats: {'backs': 0, 'time': 1.2270002365112305}
Initiating sudoku 107
Sudoku 107, stats: {'backs': 4719, 'time': 14.063001155853271}
Initiating sudoku 108
Sudoku 108, stats: {'backs': 6233, 'time': 24.600001096725464}
Initiating sudoku 109
Sudoku 109, stats: {'backs': 25, 'time': 1.3310000896453857}
Initiating sudoku 110
Sudoku 110, stats: {'backs': 307, 'time': 2.5310001373291016}
Initiating sudoku 111
Sudoku 111, stats: {'backs': 23, 'time': 1.228996992111206}
Initiating sudoku 112
Sudoku 112, stats: {'backs': 850, 'time': 4.115025520324707}
Initiating sudoku 113
Sudoku 113, stats: {'backs': 65, 'time': 1.6119964122772217}
Initiating sudoku 114
Su

Sudoku 201, stats: {'backs': 26, 'time': 1.2780017852783203}
Initiating sudoku 202
Sudoku 202, stats: {'backs': 140, 'time': 1.7140002250671387}
Initiating sudoku 203
Sudoku 203, stats: {'backs': 62, 'time': 1.4709985256195068}
Initiating sudoku 204
Sudoku 204, stats: {'backs': 12, 'time': 1.2969982624053955}
Initiating sudoku 205
Sudoku 205, stats: {'backs': 325, 'time': 2.110001564025879}
Initiating sudoku 206
Sudoku 206, stats: {'backs': 171, 'time': 1.8960018157958984}
Initiating sudoku 207
Sudoku 207, stats: {'backs': 504, 'time': 3.289998769760132}
Initiating sudoku 208
Sudoku 208, stats: {'backs': 50, 'time': 1.332007646560669}
Initiating sudoku 209
Sudoku 209, stats: {'backs': 53, 'time': 1.440990924835205}
Initiating sudoku 210
Sudoku 210, stats: {'backs': 34, 'time': 1.4639992713928223}
Initiating sudoku 211
Sudoku 211, stats: {'backs': 912, 'time': 4.6970014572143555}
Initiating sudoku 212
Sudoku 212, stats: {'backs': 55, 'time': 1.362001895904541}
Initiating sudoku 213
Sudo

Sudoku 300, stats: {'backs': 30, 'time': 1.2150001525878906}
Initiating sudoku 301
Sudoku 301, stats: {'backs': 1649, 'time': 7.880999803543091}
Initiating sudoku 302
Sudoku 302, stats: {'backs': 40, 'time': 1.2979998588562012}
Initiating sudoku 303
Sudoku 303, stats: {'backs': 213, 'time': 1.7230005264282227}
Initiating sudoku 304
Sudoku 304, stats: {'backs': 21, 'time': 1.2589988708496094}
Initiating sudoku 305
Sudoku 305, stats: {'backs': 741, 'time': 3.546001672744751}
Initiating sudoku 306
Sudoku 306, stats: {'backs': 800, 'time': 3.465019702911377}
Initiating sudoku 307
Sudoku 307, stats: {'backs': 35, 'time': 1.2609889507293701}
Initiating sudoku 308
Sudoku 308, stats: {'backs': 0, 'time': 1.3329877853393555}
Initiating sudoku 309
Sudoku 309, stats: {'backs': 85, 'time': 1.3740026950836182}
Initiating sudoku 310
Sudoku 310, stats: {'backs': 346, 'time': 2.1459999084472656}
Initiating sudoku 311
Sudoku 311, stats: {'backs': 331, 'time': 1.9839999675750732}
Initiating sudoku 312
S

Sudoku 399, stats: {'backs': 2403, 'time': 9.486002206802368}
Initiating sudoku 400
Sudoku 400, stats: {'backs': 0, 'time': 1.1919877529144287}
Initiating sudoku 401
Sudoku 401, stats: {'backs': 594, 'time': 2.6169991493225098}
Initiating sudoku 402
Sudoku 402, stats: {'backs': 90, 'time': 1.325000524520874}
Initiating sudoku 403
Sudoku 403, stats: {'backs': 111, 'time': 1.5679998397827148}
Initiating sudoku 404
Sudoku 404, stats: {'backs': 397, 'time': 2.096998691558838}
Initiating sudoku 405
Sudoku 405, stats: {'backs': 510, 'time': 2.503998279571533}
Initiating sudoku 406
Sudoku 406, stats: {'backs': 0, 'time': 1.2490031719207764}
Initiating sudoku 407
Sudoku 407, stats: {'backs': 66, 'time': 1.5350019931793213}
Initiating sudoku 408
Sudoku 408, stats: {'backs': 55, 'time': 1.3469977378845215}
Initiating sudoku 409
Sudoku 409, stats: {'backs': 218, 'time': 1.7190003395080566}
Initiating sudoku 410
Sudoku 410, stats: {'backs': 0, 'time': 1.404998540878296}
Initiating sudoku 411
Sudok

...

Sudoku 597, stats: {'backs': 365, 'time': 2.5690016746520996}
Initiating sudoku 598
Sudoku 598, stats: {'backs': 49, 'time': 1.2879974842071533}
Initiating sudoku 599
Sudoku 599, stats: {'backs': 186, 'time': 1.8560001850128174}
Initiating sudoku 600
Sudoku 600, stats: {'backs': 0, 'time': 1.2330026626586914}
Initiating sudoku 601
Sudoku 601, stats: {'backs': 6, 'time': 1.2629995346069336}
Initiating sudoku 602
Sudoku 602, stats: {'backs': 486, 'time': 2.8239986896514893}
Initiating sudoku 603
Sudoku 603, stats: {'backs': 51, 'time': 1.334000825881958}
Initiating sudoku 604
Sudoku 604, stats: {'backs': 369, 'time': 2.257997989654541}
Initiating sudoku 605
Sudoku 605, stats: {'backs': 195, 'time': 1.735002040863037}
Initiating sudoku 606
Sudoku 606, stats: {'backs': 237, 'time': 1.9520323276519775}
Initiating sudoku 607
Sudoku 607, stats: {'backs': 135, 'time': 1.585965871810913}
Initiating sudoku 608
Sudoku 608, stats: {'backs': 41, 'time': 1.292999267578125}
Initiating sudoku 609
Sudo

Sudoku 696, stats: {'backs': 1978, 'time': 8.621999502182007}
Initiating sudoku 697
Sudoku 697, stats: {'backs': 415, 'time': 2.69199800491333}
Initiating sudoku 698
Sudoku 698, stats: {'backs': 0, 'time': 1.2500033378601074}
Initiating sudoku 699
Sudoku 699, stats: {'backs': 47, 'time': 1.5269997119903564}
Initiating sudoku 700
Sudoku 700, stats: {'backs': 255, 'time': 1.9689998626708984}
Initiating sudoku 701
Sudoku 701, stats: {'backs': 1509, 'time': 5.480999708175659}
Initiating sudoku 702
Sudoku 702, stats: {'backs': 667, 'time': 3.25100040435791}
Initiating sudoku 703
Sudoku 703, stats: {'backs': 397, 'time': 2.1710000038146973}
Initiating sudoku 704
Sudoku 704, stats: {'backs': 292, 'time': 2.2530229091644287}
Initiating sudoku 705
Sudoku 705, stats: {'backs': 121, 'time': 1.4749770164489746}
Initiating sudoku 706
Sudoku 706, stats: {'backs': 5573, 'time': 19.55399751663208}
Initiating sudoku 707
Sudoku 707, stats: {'backs': 53, 'time': 1.3890013694763184}
Initiating sudoku 708


Sudoku 795, stats: {'backs': 831, 'time': 3.7349979877471924}
Initiating sudoku 796
Sudoku 796, stats: {'backs': 6656, 'time': 24.020010948181152}
Initiating sudoku 797
Sudoku 797, stats: {'backs': 277, 'time': 1.866990566253662}
Initiating sudoku 798
Sudoku 798, stats: {'backs': 5, 'time': 1.201000452041626}
Initiating sudoku 799
Sudoku 799, stats: {'backs': 161, 'time': 1.864018201828003}
Initiating sudoku 800
Sudoku 800, stats: {'backs': 140, 'time': 1.6199827194213867}
Initiating sudoku 801
Sudoku 801, stats: {'backs': 529, 'time': 2.9799904823303223}
Initiating sudoku 802
Sudoku 802, stats: {'backs': 849, 'time': 3.7930006980895996}
Initiating sudoku 803
Sudoku 803, stats: {'backs': 5030, 'time': 16.686997890472412}
Initiating sudoku 804
Sudoku 804, stats: {'backs': 107, 'time': 1.461003065109253}
Initiating sudoku 805
Sudoku 805, stats: {'backs': 357, 'time': 1.930997371673584}
Initiating sudoku 806
Sudoku 806, stats: {'backs': 250, 'time': 1.8900012969970703}
Initiating sudoku 8

Sudoku 894, stats: {'backs': 316, 'time': 2.108001708984375}
Initiating sudoku 895
Sudoku 895, stats: {'backs': 213, 'time': 2.431997299194336}
Initiating sudoku 896
Sudoku 896, stats: {'backs': 1131, 'time': 5.328002452850342}
Initiating sudoku 897
Sudoku 897, stats: {'backs': 214, 'time': 2.036998748779297}
Initiating sudoku 898
Sudoku 898, stats: {'backs': 23, 'time': 1.4000012874603271}
Initiating sudoku 899
Sudoku 899, stats: {'backs': 0, 'time': 1.3050003051757812}
Initiating sudoku 900
Sudoku 900, stats: {'backs': 111, 'time': 1.4850001335144043}
Initiating sudoku 901
Sudoku 901, stats: {'backs': 479, 'time': 2.7509994506835938}
Initiating sudoku 902
Sudoku 902, stats: {'backs': 2510, 'time': 9.911001443862915}
Initiating sudoku 903
Sudoku 903, stats: {'backs': 977, 'time': 3.806999683380127}
Initiating sudoku 904
Sudoku 904, stats: {'backs': 2990, 'time': 13.665997505187988}
Initiating sudoku 905
Sudoku 905, stats: {'backs': 851, 'time': 4.229001760482788}
Initiating sudoku 906

Sudoku 993, stats: {'backs': 191, 'time': 1.5300002098083496}
Initiating sudoku 994
Sudoku 994, stats: {'backs': 697, 'time': 3.468996286392212}
Initiating sudoku 995
Sudoku 995, stats: {'backs': 91, 'time': 1.3590004444122314}
Initiating sudoku 996
Sudoku 996, stats: {'backs': 120, 'time': 1.4810144901275635}
Initiating sudoku 997
Sudoku 997, stats: {'backs': 122, 'time': 1.6489884853363037}
Initiating sudoku 998
Sudoku 998, stats: {'backs': 62, 'time': 1.300997257232666}


In [31]:
test_random_1000 = []
heuristics = ['1']
for heuristic in heuristics:
    for sudoku_nr in range(999):
        stats = {"backs":0, "time":0}
        print(f'Initiating sudoku {sudoku_nr}')
        start_time = time.time()
        run_sat(heuristic,sudoku_nr,'sudokus/1000 sudokus.txt')
        stop_time = time.time()
        stats['time'] = stop_time - start_time
        test_random_1000.append([sudoku_nr,'random',stats['time'],stats['backs']])
        print(f'Sudoku {sudoku_nr}, stats: {stats}')

Initiating sudoku 0
Sudoku 0, stats: {'backs': 146, 'time': 2.218323230743408}
Initiating sudoku 1
Sudoku 1, stats: {'backs': 179, 'time': 3.148998737335205}
Initiating sudoku 2
Sudoku 2, stats: {'backs': 194, 'time': 3.0699985027313232}
Initiating sudoku 3
Sudoku 3, stats: {'backs': 90, 'time': 1.761000156402588}
Initiating sudoku 4
Sudoku 4, stats: {'backs': 260, 'time': 2.743002414703369}
Initiating sudoku 5
Sudoku 5, stats: {'backs': 73, 'time': 1.6730000972747803}
Initiating sudoku 6
Sudoku 6, stats: {'backs': 72, 'time': 2.7170000076293945}
Initiating sudoku 7
Sudoku 7, stats: {'backs': 21, 'time': 2.9269979000091553}
Initiating sudoku 8
Sudoku 8, stats: {'backs': 25, 'time': 1.4579992294311523}
Initiating sudoku 9
Sudoku 9, stats: {'backs': 238, 'time': 2.5250024795532227}
Initiating sudoku 10
Sudoku 10, stats: {'backs': 209, 'time': 2.927000045776367}
Initiating sudoku 11
Sudoku 11, stats: {'backs': 40, 'time': 1.4749979972839355}
Initiating sudoku 12
Sudoku 12, stats: {'backs'

Sudoku 102, stats: {'backs': 75, 'time': 2.695000171661377}
Initiating sudoku 103
Sudoku 103, stats: {'backs': 31, 'time': 2.4610023498535156}
Initiating sudoku 104
Sudoku 104, stats: {'backs': 50, 'time': 3.084000587463379}
Initiating sudoku 105
Sudoku 105, stats: {'backs': 24, 'time': 1.495997667312622}
Initiating sudoku 106
Sudoku 106, stats: {'backs': 13, 'time': 2.3749992847442627}
Initiating sudoku 107
Sudoku 107, stats: {'backs': 293, 'time': 2.202003002166748}
Initiating sudoku 108
Sudoku 108, stats: {'backs': 265, 'time': 3.910999059677124}
Initiating sudoku 109
Sudoku 109, stats: {'backs': 9, 'time': 1.189000129699707}
Initiating sudoku 110
Sudoku 110, stats: {'backs': 46, 'time': 1.3529982566833496}
Initiating sudoku 111
Sudoku 111, stats: {'backs': 8, 'time': 2.3159995079040527}
Initiating sudoku 112
Sudoku 112, stats: {'backs': 179, 'time': 1.9530034065246582}
Initiating sudoku 113
Sudoku 113, stats: {'backs': 76, 'time': 2.7029967308044434}
Initiating sudoku 114
Sudoku 11

Sudoku 201, stats: {'backs': 66, 'time': 2.751997232437134}
Initiating sudoku 202
Sudoku 202, stats: {'backs': 12, 'time': 1.4250023365020752}
Initiating sudoku 203
Sudoku 203, stats: {'backs': 37, 'time': 2.434999704360962}
Initiating sudoku 204
Sudoku 204, stats: {'backs': 41, 'time': 2.4839978218078613}
Initiating sudoku 205
Sudoku 205, stats: {'backs': 39, 'time': 2.4559998512268066}
Initiating sudoku 206
Sudoku 206, stats: {'backs': 21, 'time': 2.416001558303833}
Initiating sudoku 207
Sudoku 207, stats: {'backs': 184, 'time': 2.1270012855529785}
Initiating sudoku 208
Sudoku 208, stats: {'backs': 46, 'time': 1.3749973773956299}
Initiating sudoku 209
Sudoku 209, stats: {'backs': 51, 'time': 1.4890007972717285}
Initiating sudoku 210
Sudoku 210, stats: {'backs': 25, 'time': 1.5530014038085938}
Initiating sudoku 211
Sudoku 211, stats: {'backs': 41, 'time': 2.534000873565674}
Initiating sudoku 212
Sudoku 212, stats: {'backs': 39, 'time': 1.46799635887146}
Initiating sudoku 213
Sudoku 21

Sudoku 301, stats: {'backs': 280, 'time': 4.0639967918396}
Initiating sudoku 302
Sudoku 302, stats: {'backs': 45, 'time': 1.5600011348724365}
Initiating sudoku 303
Sudoku 303, stats: {'backs': 21, 'time': 1.339998722076416}
Initiating sudoku 304
Sudoku 304, stats: {'backs': 56, 'time': 2.53200364112854}
Initiating sudoku 305
Sudoku 305, stats: {'backs': 103, 'time': 1.4929962158203125}
Initiating sudoku 306
Sudoku 306, stats: {'backs': 114, 'time': 2.9590024948120117}
Initiating sudoku 307
Sudoku 307, stats: {'backs': 43, 'time': 2.4379889965057373}
Initiating sudoku 308
Sudoku 308, stats: {'backs': 10, 'time': 1.2429940700531006}
Initiating sudoku 309
Sudoku 309, stats: {'backs': 26, 'time': 2.4180009365081787}
Initiating sudoku 310
Sudoku 310, stats: {'backs': 11, 'time': 1.2130019664764404}
Initiating sudoku 311
Sudoku 311, stats: {'backs': 97, 'time': 1.617997169494629}
Initiating sudoku 312
Sudoku 312, stats: {'backs': 27, 'time': 1.4090027809143066}
Initiating sudoku 313
Sudoku 3

Sudoku 401, stats: {'backs': 125, 'time': 1.6639924049377441}
Initiating sudoku 402
Sudoku 402, stats: {'backs': 28, 'time': 2.4599997997283936}
Initiating sudoku 403
Sudoku 403, stats: {'backs': 21, 'time': 2.3260011672973633}
Initiating sudoku 404
Sudoku 404, stats: {'backs': 71, 'time': 1.460998773574829}
Initiating sudoku 405
Sudoku 405, stats: {'backs': 215, 'time': 2.8619980812072754}
Initiating sudoku 406
Sudoku 406, stats: {'backs': 12, 'time': 2.3810136318206787}
Initiating sudoku 407
Sudoku 407, stats: {'backs': 44, 'time': 2.393988609313965}
Initiating sudoku 408
Sudoku 408, stats: {'backs': 36, 'time': 1.3799974918365479}
Initiating sudoku 409
Sudoku 409, stats: {'backs': 218, 'time': 1.9640021324157715}
Initiating sudoku 410
Sudoku 410, stats: {'backs': 37, 'time': 1.494999885559082}
Initiating sudoku 411
Sudoku 411, stats: {'backs': 35, 'time': 2.412001371383667}
Initiating sudoku 412
Sudoku 412, stats: {'backs': 58, 'time': 1.550001621246338}
Initiating sudoku 413
Sudoku

...

Sudoku 599, stats: {'backs': 45, 'time': 2.5460004806518555}
Initiating sudoku 600
Sudoku 600, stats: {'backs': 32, 'time': 2.5510005950927734}
Initiating sudoku 601
Sudoku 601, stats: {'backs': 54, 'time': 1.562002182006836}
Initiating sudoku 602
Sudoku 602, stats: {'backs': 75, 'time': 1.6069974899291992}
Initiating sudoku 603
Sudoku 603, stats: {'backs': 19, 'time': 1.251002550125122}
Initiating sudoku 604
Sudoku 604, stats: {'backs': 42, 'time': 2.489999294281006}
Initiating sudoku 605
Sudoku 605, stats: {'backs': 29, 'time': 2.63799786567688}
Initiating sudoku 606
Sudoku 606, stats: {'backs': 332, 'time': 2.6810014247894287}
Initiating sudoku 607
Sudoku 607, stats: {'backs': 23, 'time': 1.3650097846984863}
Initiating sudoku 608
Sudoku 608, stats: {'backs': 11, 'time': 2.4839885234832764}
Initiating sudoku 609
Sudoku 609, stats: {'backs': 13, 'time': 1.268003225326538}
Initiating sudoku 610
Sudoku 610, stats: {'backs': 49, 'time': 2.499013662338257}
Initiating sudoku 611
Sudoku 611

Sudoku 698, stats: {'backs': 5, 'time': 2.2919998168945312}
Initiating sudoku 699
Sudoku 699, stats: {'backs': 39, 'time': 1.5650031566619873}
Initiating sudoku 700
Sudoku 700, stats: {'backs': 64, 'time': 2.563997745513916}
Initiating sudoku 701
Sudoku 701, stats: {'backs': 36, 'time': 2.422999143600464}
Initiating sudoku 702
Sudoku 702, stats: {'backs': 64, 'time': 1.4740002155303955}
Initiating sudoku 703
Sudoku 703, stats: {'backs': 15, 'time': 1.2930028438568115}
Initiating sudoku 704
Sudoku 704, stats: {'backs': 10, 'time': 1.4300003051757812}
Initiating sudoku 705
Sudoku 705, stats: {'backs': 63, 'time': 1.465996503829956}
Initiating sudoku 706
Sudoku 706, stats: {'backs': 735, 'time': 4.656001091003418}
Initiating sudoku 707
Sudoku 707, stats: {'backs': 13, 'time': 1.3859989643096924}
Initiating sudoku 708
Sudoku 708, stats: {'backs': 179, 'time': 2.9090023040771484}
Initiating sudoku 709
Sudoku 709, stats: {'backs': 12, 'time': 1.3629982471466064}
Initiating sudoku 710
Sudoku 

Sudoku 797, stats: {'backs': 16, 'time': 1.287998914718628}
Initiating sudoku 798
Sudoku 798, stats: {'backs': 15, 'time': 2.5099990367889404}
Initiating sudoku 799
Sudoku 799, stats: {'backs': 133, 'time': 1.6629996299743652}
Initiating sudoku 800
Sudoku 800, stats: {'backs': 67, 'time': 2.5370023250579834}
Initiating sudoku 801
Sudoku 801, stats: {'backs': 13, 'time': 1.334007740020752}
Initiating sudoku 802
Sudoku 802, stats: {'backs': 114, 'time': 2.744990348815918}
Initiating sudoku 803
Sudoku 803, stats: {'backs': 112, 'time': 1.7800312042236328}
Initiating sudoku 804
Sudoku 804, stats: {'backs': 58, 'time': 1.3829703330993652}
Initiating sudoku 805
Sudoku 805, stats: {'backs': 46, 'time': 1.3469972610473633}
Initiating sudoku 806
Sudoku 806, stats: {'backs': 48, 'time': 1.6890032291412354}
Initiating sudoku 807
Sudoku 807, stats: {'backs': 10, 'time': 1.2836205959320068}
Initiating sudoku 808
Sudoku 808, stats: {'backs': 122, 'time': 2.9531030654907227}
Initiating sudoku 809
Sud

Sudoku 896, stats: {'backs': 20, 'time': 1.2949984073638916}
Initiating sudoku 897
Sudoku 897, stats: {'backs': 6, 'time': 2.291999340057373}
Initiating sudoku 898
Sudoku 898, stats: {'backs': 46, 'time': 1.4540014266967773}
Initiating sudoku 899
Sudoku 899, stats: {'backs': 140, 'time': 2.786998987197876}
Initiating sudoku 900
Sudoku 900, stats: {'backs': 50, 'time': 1.375002145767212}
Initiating sudoku 901
Sudoku 901, stats: {'backs': 136, 'time': 1.758997917175293}
Initiating sudoku 902
Sudoku 902, stats: {'backs': 30, 'time': 1.568002462387085}
Initiating sudoku 903
Sudoku 903, stats: {'backs': 1, 'time': 1.242997407913208}
Initiating sudoku 904
Sudoku 904, stats: {'backs': 161, 'time': 3.1020026206970215}
Initiating sudoku 905
Sudoku 905, stats: {'backs': 113, 'time': 1.857018232345581}
Initiating sudoku 906
Sudoku 906, stats: {'backs': 100, 'time': 1.6099793910980225}
Initiating sudoku 907
Sudoku 907, stats: {'backs': 28, 'time': 1.2810020446777344}
Initiating sudoku 908
Sudoku 9

Sudoku 995, stats: {'backs': 4, 'time': 2.3380000591278076}
Initiating sudoku 996
Sudoku 996, stats: {'backs': 60, 'time': 2.564000129699707}
Initiating sudoku 997
Sudoku 997, stats: {'backs': 63, 'time': 1.5976498126983643}
Initiating sudoku 998
Sudoku 998, stats: {'backs': 45, 'time': 2.6919987201690674}


In [30]:
df_1 = pd.DataFrame(test_mrv_1000)
df_1.to_csv('test_mrv_1000.csv')
df_2 = pd.DataFrame(test_jw_1000)
df_2.to_csv('test_jw_1000.csv')

In [32]:
df_1 = pd.DataFrame(test_random_1000)
df_1.to_csv('test_random_1000.csv')